In [1]:
import MySQLdb
import csv
import os
import subprocess
import pandas as pd

In [2]:
def write_header():
    head = ["cid","time","compile","vmlinux","GZIP-bzImage","GZIP-vmlinux","GZIP","BZIP2-bzImage","BZIP2-vmlinux","BZIP2","LZMA-bzImage","LZMA-vmlinux","LZMA","XZ-bzImage","XZ-vmlinux","XZ","LZO-bzImage","LZO-vmlinux","LZO","LZ4-bzImage","LZ4-vmlinux","LZ4","basic-vmlinux","basic-GZIP-bzImage","basic-GZIP-vmlinux","basic-GZIP","basic-BZIP2-bzImage","basic-BZIP2-vmlinux","basic-BZIP2","basic-LZMA-bzImage","basic-LZMA-vmlinux","basic-LZMA","basic-XZ-bzImage","basic-XZ-vmlinux","basic-XZ","basic-LZO-bzImage","basic-LZO-vmlinux","basic-LZO","basic-LZ4-bzImage","basic-LZ4-vmlinux","basic-LZ4"]
    with open("stats.csv", 'w') as file:
        writer = csv.DictWriter(file,head)
        writer.writeheader()

In [3]:
def kernel2csv(compressed):
        compressed = compressed.split(" , ")
        parse_array = [i.split(" : ") for i in compressed]
        final = [fin[1] for fin in parse_array]

        return final

In [4]:
def compute_rows():
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT cid, compilation_time, core_size, compressed_sizes FROM Compilations WHERE cid >= 20000 ORDER BY cid DESC"
        cursor.execute(query)
        entry = cursor.fetchall()
    except:
        print("Error")
    finally:
        cursor.close()
        socket.close()

    return entry

In [5]:
## Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [6]:
def create_csv():
    
    subprocess.run("rm stats.csv", shell=True)
    if not os.path.exists("stats.csv"):
        write_header()  
        
    entry_unformatted = compute_rows()
    print(entry_unformatted[3], flush=True)
    compr = kernel2csv(entry_unformatted[3])
    entry = entry_unformatted[0: 3].append(compr)
    
    with open("stats.csv", 'a') as file:
        writer = csv.writer(file)
        for line in entry:
            writer.writerow(line)
    subprocess.run('wc -c stats.csv', shell=True)
    print("Ok", flush=True)
    

In [7]:
def main():  
    create_csv()
    min_max()
    

In [8]:
def min_max():
    df = pd.read_csv('stats.csv')
    print("Time Max:", max(df["time"]), end="")
    print(" Time Min:", min(df["time"]))
    print("Size Max:", max(df["vmlinux"]), end="")
    print(" Size Min:", min(df["vmlinux"]))
        

In [9]:
main()

(56265, 369.792, 61867256, 'GZIP-bzImage : 16883152 , GZIP-vmlinux : 18970064 , GZIP : 16737559 , BZIP2-bzImage : 19951056 , BZIP2-vmlinux : 22037416 , BZIP2 : 15686470 , LZMA-bzImage : 13376976 , LZMA-vmlinux : 15463376 , LZMA : 13240423 , XZ-bzImage : 11058640 , XZ-vmlinux : 13145464 , XZ : 10914096 , LZO-bzImage : 18611664 , LZO-vmlinux : 20697976 , LZO : 18476033 , LZ4-bzImage : 19934672 , LZ4-vmlinux : 22021296 , LZ4 : 19790730')


AttributeError: 'tuple' object has no attribute 'split'